# Notebook 3.2: Do FBA, FVA and flux sampling for each of the GSMMs. Models with deletions and all SNPs

In [34]:
import os
import cobra
import pandas as pd
from os.path import join
import numpy as np
from cobra.flux_analysis import flux_variability_analysis
from cobra.sampling import sample
import copy as cp
from cobra.medium import minimal_medium

Directory stuff...

In [35]:
# Directory stuff
mod_dir = '../results/built_models/delsAllsnps_mods/cur/'
baseMod_dir = '../data/models/'
delMatFile = '../results/del_AllSNPs_mat_cur.csv'
res_dir = '../results/simulations/delsAllSNPs/'
FBA_dir = res_dir + 'FBA/'
FVA_dir = res_dir + 'FVA/'
sampling_dir = res_dir + 'sampling/'

In [36]:
# Create directories if they don't exist
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
if not os.path.exists(FBA_dir):
    os.makedirs(FBA_dir)
if not os.path.exists(FVA_dir):
    os.makedirs(FVA_dir)
if not os.path.exists(sampling_dir):
    os.makedirs(sampling_dir)

### Load base reconstruction, built models, different medium compositions and curated delMat.csv for start running simulations

In [37]:
base_mod = cobra.io.load_json_model(join(baseMod_dir, "iEK1011_2.0.json"))

In [5]:
# Create 7H9 OADC medium, obtained from S8 file matlab scripts from iEK1011 2.0 paper
mi7H9_OADC_Med = {'EX_glu__L_e':1,'EX_cu2_e':1000,'EX_btn_e':1,'EX_pydxn_e':1,'EX_ca2_e':1000, 'EX_mg2_e':1000, 'EX_h_e':1000, 'EX_k_e':1000,
'EX_nh4_e':10, 'EX_h2o_e':1000,'EX_pi_e':1, 'EX_cl_e':1000, 'EX_o2_e':20,
'EX_na1_e':1000, 'EX_so4_e':1000,'EX_cit_e':1,'EX_fe3_e':5, 'EX_glyc_e':1,
'EX_glc__D_e':1,'EX_ocdca_e':1}

In [6]:
# Create 7H9 OADC supplemented with cholesterol
mi7H9_OADCchol_Med = cp.copy(mi7H9_OADC_Med)
mi7H9_OADCchol_Med['EX_chsterol_e'] = 1

In [7]:
# Load different iEK1011 models (in different mediums) to retrieve mediums 
inVivoMed = cobra.io.load_json_model('C:/Users/Guillem/Documents/PhD/comput/models/MTBC/iEK1011_inVivo_media.json').medium
deJesuMed = cobra.io.load_json_model('C:/Users/Guillem/Documents/PhD/comput/models/MTBC/iEK1011_deJesusEssen_media.json').medium
drugTsMed = cobra.io.load_json_model('C:/Users/Guillem/Documents/PhD/comput/models/MTBC/iEK1011_drugTesting_media.json').medium
grifEsMed = cobra.io.load_json_model('C:/Users/Guillem/Documents/PhD/comput/models/MTBC/iEK1011_griffinEssen_media.json').medium
mi7H10Med = cobra.io.load_json_model('C:/Users/Guillem/Documents/PhD/comput/models/MTBC/iEK1011_m7H10_media.json').medium

In [8]:
grifEsMed

{'EX_asn_L': 1.0,
 'EX_ca2': 1000.0,
 'EX_cit': 1.0,
 'EX_cl': 1000.0,
 'EX_fe3': 5.0,
 'EX_glyc': 1.0,
 'EX_h2o': 1000.0,
 'EX_h': 1000.0,
 'EX_mg2': 1000.0,
 'EX_nh4': 10.0,
 'EX_o2': 20.0,
 'EX_pi': 1.0,
 'EX_so4': 1000.0,
 'EX_chsterol': 1.0,
 'EX_etoh': 1.0}

In [9]:
# Exchange reactions have different names in old model than in new, so change the keys to make mediums compatible with models. 

for m in list(inVivoMed.keys()):
    if 'L' in m or 'D' in m:
        k = m[0:(len(m)-1)] + '_' + m[len(m)-1] + '_e'
    else:
        k = m + '_e'
    inVivoMed[k] = inVivoMed.pop(m)

for m in list(deJesuMed.keys()):
    if 'L' in m or 'D' in m:
        k = m[0:(len(m)-1)] + '_' + m[len(m)-1] + '_e'
    else:
        k = m + '_e'
    deJesuMed[k] = deJesuMed.pop(m)

for m in list(drugTsMed.keys()):
    if 'L' in m or 'D' in m:
        k = m[0:(len(m)-1)] + '_' + m[len(m)-1] + '_e'
    else:
        k = m + '_e'
    drugTsMed[k] = drugTsMed.pop(m)

for m in list(grifEsMed.keys()):
    if 'L' in m or 'D' in m:
        k = m[0:(len(m)-1)] + '_' + m[len(m)-1] + '_e'
    else:
        k = m + '_e'
    grifEsMed[k] = grifEsMed.pop(m)

for m in list(mi7H10Med.keys()):
    if 'L' in m or 'D' in m:
        k = m[0:(len(m)-1)] + '_' + m[len(m)-1] + '_e'
    else:
        k = m + '_e'
    mi7H10Med[k] = mi7H10Med.pop(m)

In [10]:
delMat = pd.read_csv(delMatFile, index_col = 0)

In [11]:
#load the draft models created from Notebook2
model_files=os.listdir(mod_dir)

In [12]:
#Load each target lineage model, initialize it to Middlebrock 7H10 media and check if the models can optimize for
# biomass production
for model in model_files:
    mod=cobra.io.load_json_model(mod_dir + model)
    mod.medium = mi7H9_OADC_Med
    mod.objective = 'BIOMASS__2'
    print (mod.id, mod.slim_optimize())

A1 0.35770769901798255
A2 0.36832917643404783
A3 0.3549438657024325
A4 0.3683231557178398
L1 0.3683291764340539
L2 0.36832917643404806
L3 0.36730315497551447
L4 0.3683291764340529
L5 0.36832917643405394
L6 0.368329176434051
L7 0.21285549166171763
L8 0.3617057204864494
L9 0.36832917643405044


Everything is fine.

## Analysis

In [13]:
L1 = cobra.io.load_json_model(join(mod_dir, "L1_cur.json"))
L2 = cobra.io.load_json_model(join(mod_dir, "L2_cur.json"))
L3 = cobra.io.load_json_model(join(mod_dir, "L3_cur.json"))
L4 = cobra.io.load_json_model(join(mod_dir, "L4_cur.json"))
L5 = cobra.io.load_json_model(join(mod_dir, "L5_cur.json"))
L6 = cobra.io.load_json_model(join(mod_dir, "L6_cur.json"))
L7 = cobra.io.load_json_model(join(mod_dir, "L7_cur.json"))
L8 = cobra.io.load_json_model(join(mod_dir, "L8_cur.json"))
L9 = cobra.io.load_json_model(join(mod_dir, "L9_cur.json"))
A1 = cobra.io.load_json_model(join(mod_dir, "A1_cur.json"))
A2 = cobra.io.load_json_model(join(mod_dir, "A2_cur.json"))
A3 = cobra.io.load_json_model(join(mod_dir, "A3_cur.json"))
A4 = cobra.io.load_json_model(join(mod_dir, "A4_cur.json"))

In [14]:
defModLst = [L1, L2, L3, L4, L5, L6, L7, L8, L9, A1, A2, A3, A4]

### Build list and DF of removed reactions

In [16]:
iEK1011_rxns = [x.id for x in base_mod.reactions]

In [17]:
rmRxnsTab = pd.DataFrame(index = iEK1011_rxns, columns = list(delMat.columns))

In [18]:
allRemRxns = []
for mod in defModLst:
    lin = mod.id
    modRxns = [x.id for x in mod.reactions]
    remRxnsLin = [x for x in iEK1011_rxns if x not in modRxns]
    allRemRxns = [*allRemRxns, *remRxnsLin]
    remRxnList = []
    for idx in list(rmRxnsTab.index):
        if idx in remRxnsLin:
            remRxnList.append(0.0)
        else:
            remRxnList.append(1.0)
    rmRxnsTab[lin] = remRxnList
allRemRxns = list(np.unique(allRemRxns))

In [19]:
rmRxnsTab = rmRxnsTab.loc[[sum(rmRxnsTab.loc[x] == 1) != rmRxnsTab.shape[1] for x in rmRxnsTab.index]]

### Minimal medium determination

In [20]:
minMedDF = pd.DataFrame(index = list(minimal_medium(L7, 0.2128).keys()), columns = [x.id for x in defModLst])

In [21]:
for mod in defModLst:
    lin = mod.id
    minMed = minimal_medium(mod, 0.2128)
    for met in minMed.keys():
        value = minMed.loc[met]
        minMedDF[lin][met] = value

In [22]:
minMedDF[pd.isna(minMedDF)] = 0

In [23]:
minMedDF.to_csv(res_dir + "minMeds.csv")

### FBA

In [24]:
defModLst[0].id

'L1'

In [25]:
def doFBA(medium = mi7H9_OADC_Med):
    for mod in defModLst:
        mod.medium = medium
    solDict = {}
    lins = [x.id for x in defModLst]
    for i in range(0, len(defModLst)):
        lin = lins[i]
        sol = defModLst[i].optimize().fluxes
        solDict[lin] = sol
    solDF = pd.DataFrame(index = lins, columns = [x.id for x in base_mod.reactions])
    for l in lins:
        solFlxs = solDict[l]
        for r in list(solDF.columns):
            if r in list(solFlxs.index):
                solDF[r][l] = solDict[l][r]
            else:
                solDF[r][l] = 0.0
    return solDF

In [14]:
# Initialize model in 7H10 medium
L1.medium = mi7H9_OADC_Med
L2.medium = mi7H9_OADC_Med
L3.medium = mi7H9_OADC_Med
L4.medium = mi7H9_OADC_Med
L5.medium = mi7H9_OADC_Med
L5.medium = mi7H9_OADC_Med
L6.medium = mi7H9_OADC_Med
L7.medium = mi7H9_OADC_Med
L8.medium = mi7H9_OADC_Med
L9.medium = mi7H9_OADC_Med
A1.medium = mi7H9_OADC_Med
A2.medium = mi7H9_OADC_Med
A3.medium = mi7H9_OADC_Med
A4.medium = mi7H9_OADC_Med

In [16]:
solDict = {}
lins = [x.id for x in defModLst]
for i in range(0, len(defModLst)):
    lin = lins[i]
    sol = defModLst[i].optimize().fluxes
    solDict[lin] = sol

Convert the dictionary of solutions into a dataframe. If a reaction is not present in a model assign the flux of that column to zero. 

In [17]:
solDF = pd.DataFrame(index = lins, columns = [x.id for x in base_mod.reactions])

In [18]:
for l in lins:
    solFlxs = solDict[l]
    for r in list(solDF.columns):
        if r in list(solFlxs.index):
            solDF[r][l] = solDict[l][r]
        else:
            solDF[r][l] = 0.0

In [19]:
solDF

,ABTA,AGMT,ALAD_L,ARGDC,ARGSL,ARGSS,ASNN,ASNS1,ASP1DC,ASPO5,...,BACA,SDH1,ATPM,DHFR_copy1,BiomassGrowth_sMtb,BIOMASS__2,BIOMASS__2.1,QRr,NADH2r,ADK2
L1,0.0,0.0,0.0,0.0,0.044354,0.044354,0.0,0.017569,0.007219,0.010608,...,0.0,10.902099,0.0,0.000295,0.0,0.368329,0.0,15.381292,0.0,0.0
L2,0.0,0.0,0.0,0.0,0.044354,0.044354,0.0,0.017569,0.007219,0.010608,...,0.0,10.902099,0.0,0.000295,0.0,0.368329,0.0,14.909201,0.0,0.0
L3,0.0,0.0,0.0,0.0,0.044231,0.044231,0.0,0.01752,0.007199,0.010578,...,0.0,10.938085,0.0,0.000294,0.0,0.367303,0.0,15.321091,0.0,0.0
L4,0.0,0.0,0.0,0.0,0.044354,0.044354,0.0,0.017569,0.007219,0.010608,...,0.0,10.902099,0.0,0.000295,0.0,0.368329,0.0,15.381292,0.0,0.0
L5,0.0,0.0,0.0,0.0,0.044354,0.044354,0.0,0.017569,0.007219,0.010608,...,0.0,10.902099,0.0,0.000295,0.0,0.368329,0.0,15.381292,0.0,0.0
L6,0.0,0.0,0.0,0.0,0.044354,0.044354,0.0,0.017569,0.007219,0.010608,...,0.0,10.902099,0.0,0.000295,0.0,0.368329,0.0,14.909201,13.224858,0.0
L7,0.0,0.0,0.0,0.0,0.025632,0.025632,0.0,0.010153,0.004172,0.00613,...,0.0,3.7019,0.0,0.00017,0.0,0.212855,0.0,7.154628,0.0,0.0
L8,0.0,0.0,0.0,0.0,0.043557,0.043557,0.0,0.017253,0.007089,0.010417,...,0.0,10.594621,0.0,0.000289,0.0,0.361706,0.0,16.199269,0.0,0.0
L9,0.0,0.0,0.0,0.0,0.044354,0.044354,0.0,0.017569,0.007219,0.010608,...,0.0,10.902099,0.0,0.000295,0.0,0.368329,0.0,15.432133,0.0,0.0
A1,0.0,0.0,0.0,0.0,0.043075,0.043075,0.0,0.017062,0.007011,0.010302,...,0.0,10.797611,0.0,0.000286,0.0,0.357708,0.0,15.068295,0.0,0.0


In [20]:
# Create output path for 7H9 OADC FBA simulation if it doesn't exist
FBA_mi7H9OADCDir = FBA_dir + 'mi7H9OADCMed/'
if not os.path.exists(FBA_mi7H9OADCDir):
    os.mkdir(FBA_mi7H10Dir)

In [21]:
solDF.to_csv(FBA_mi7H9OADCDir + 'fbaFlxs_7H9OADC.csv')

Get reactions of the genes that have been removed from each model and remove them from FBA Dataframe, to keep only effects propagated through network. 

In [15]:
rxnsRemoved = []
for gene in delMat.index:
    if 0 in delMat.loc[gene, ].tolist():
        rxns = list(base_mod.genes.get_by_id(gene).reactions)
        for rxn in rxns:
            rxnsRemoved.append(rxn.id)
rxnsRemoved = list(np.unique(rxnsRemoved))

In [16]:
rxnsRemoved

['ACBIPGT',
 'ACCD3',
 'ACGS',
 'ACLS',
 'ACONMT',
 'ACS',
 'ACS2',
 'ADCPS2',
 'ADCYRS',
 'ADNCYC',
 'ADOCBIK',
 'ADOCBLabc',
 'AGDC',
 'AGPAT160',
 'AGPAT160190',
 'AGPAT190',
 'AHCYSNS',
 'AHSERL2',
 'AKGDa',
 'ALCD2ir',
 'ALDD1',
 'ALDD2x',
 'AMID4',
 'AOXSr',
 'APRAUR',
 'ARGDC',
 'ASNt2r',
 'ASPt2r',
 'BALAt2r',
 'BETALDHx',
 'BTS2',
 'BTS3r',
 'CAATPS',
 'CAt4',
 'CBIAT',
 'CHLabc',
 'CHLabc_rev',
 'CHOLK_1',
 'CHORM',
 'CHSTEROLt_1',
 'CITL',
 'COCHL',
 'COCO2',
 'CPC6R',
 'CPMPS',
 'CRNabc',
 'CS',
 'CYANST',
 'CYP1',
 'CYP2',
 'CYP3',
 'CYRDAAT',
 'CYSK2_I',
 'CYSS',
 'CYSTL',
 'CYST_II',
 'Clt',
 'Cuabc',
 'Cut1',
 'DAGK160',
 'DALAt2r',
 'DCTPD',
 'DCTPD2',
 'DESAT18_1',
 'DGC3D',
 'DHPPDA',
 'DHPPDA2',
 'DMATT',
 'DSERt2',
 'DTBTST',
 'DXPS',
 'ECHA191',
 'ECHA192',
 'ECOAH12',
 'ECOAH9ir',
 'ENO',
 'EPH',
 'FACOAE140_1',
 'FACOAE160',
 'FACOAE180',
 'FACOAL140_1',
 'FACOAL160',
 'FACOAL161',
 'FACOAL180',
 'FACOAL181_2',
 'FACOAL200',
 'FACOAL80',
 'FACOALPREPH',
 'FADA6'

In [24]:
solDF_altRXNsOut = solDF.loc[:,[x not in rxnsRemoved for x in solDF.columns]]

In [25]:
solDF_altRXNsOut.to_csv(FBA_mi7H10Dir + 'fbaFlxs_7H9OADC_noRemRXNs.csv')

In [27]:
solDF_7h9OADC_chol = doFBA(mi7H9_OADCchol_Med)

In [32]:
# Create output path for 7H9 OADC chol FBA simulation if it doesn't exist
FBA_mi7H9OADCcholDir = FBA_dir + 'mi7H9OADCCholMed/'
if not os.path.exists(FBA_mi7H9OADCcholDir):
    os.mkdir(FBA_mi7H9OADCcholDir)

In [33]:
solDF_7h9OADC_chol.to_csv(FBA_mi7H9OADCcholDir + 'fbaFlxs_7H9OADCChol.csv')

### FVA

In [26]:
#FVA_mi7H10Med = {}
#for i in defModLst:
#    FVA = flux_variability_analysis(i)
#    FVA['diff'] = FVA['minimum'] - FVA['maximum']
#    FVA_mi7H10Med[i.id] = FVA

### Flux Sampling

In [17]:
samp_mi7H10Dir = sampling_dir + 'mi7H10Med/'
if not os.path.exists(samp_mi7H10Dir):
    os.mkdir(samp_mi7H10Dir)

In [18]:
samp_mi7H9OADCCholDir = sampling_dir + 'mi7H9OADCChol/'
if not os.path.exists(samp_mi7H9OADCCholDir):
    os.mkdir(samp_mi7H9OADCCholDir)

In [30]:
def sampleFluxes(modLst, outDir, size = 100, outCsv = False, medium = mi7H10Med):
    lins = [x.id for x in defModLst]
    #sampFluxDict = {}
    sampFiles = os.listdir(outDir)
    for i in range(0, len(lins)):
        lin = lins[i]
        fileName = lin + '_samp.csv'
        if fileName not in sampFiles:
            mod = modLst[i].copy()
            mod.medium = medium
            print('Sampling %s model...'%lin)
            sampFlux = sample(mod, size)
        else: 
            print('%s already sampled, loading %s'%(lin, fileName))
            sampFlux = pd.read_csv(outDir + fileName, index_col = 0)
        #sampFluxDict[lin] = sampFlux
        if outCsv == True and fileName not in sampFiles:
            sampFlux.to_csv(outDir + lin + '_samp.csv')
            print('%s_samp.csv saved at %s.'%(lin, outDir))
    #return sampFluxDict

In [ ]:
sampFluxDict_mi7H10Med = sampleFluxes(defModLst, size = 5000, outCsv = True, outDir = samp_mi7H9OADCCholDir, medium = mi7H9_OADCchol_Med)

Sampling L1 model...
L1_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L2 model...
L2_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L3 model...
L3_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L4 model...
L4_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L5 model...
L5_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L6 model...
L6_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L7 model...
L7_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L8 model...
L8_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling L9 model...
L9_samp.csv saved at ../results/simulations/delsAllSNPs/sampling/mi7H9OADCChol/.
Sampling A1 model...


In [29]:
A1_mod = defModLst[9].copy()

In [30]:
A1_mod

Name,A1
Memory address,0x01b6779b4d30
Number of metabolites,977
Number of reactions,1219
Number of groups,0
Objective expression,1.0*BIOMASS__2 - 1.0*BIOMASS__2_reverse_ecbec
Compartments,"c, e"


In [31]:
A1_mod.medium = mi7H9_OADCchol_Med

In [32]:
A1_sampFlux = sample(A1_mod, 5000)

In [33]:
A1_sampFlux.to_csv(samp_mi7H9OADCCholDir + 'A1' + '_cur.csv')

In [29]:
A2_mod = defModLst[10].copy()

In [30]:
A2_mod

Name,A2
Memory address,0x021d5a180d00
Number of metabolites,977
Number of reactions,1232
Number of groups,0
Objective expression,1.0*BIOMASS__2 - 1.0*BIOMASS__2_reverse_ecbec
Compartments,"c, e"


In [31]:
A2_mod.medium = mi7H9_OADCchol_Med

In [32]:
A2_sampFlux = sample(A2_mod, 5000)

In [33]:
A2_sampFlux.to_csv(samp_mi7H9OADCCholDir + 'A2' + '_cur.csv')

In [19]:
A3_mod = defModLst[11].copy()

In [20]:
A3_mod

Name,A3
Memory address,0x01b66f603cd0
Number of metabolites,977
Number of reactions,1194
Number of groups,0
Objective expression,1.0*BIOMASS__2 - 1.0*BIOMASS__2_reverse_ecbec
Compartments,"c, e"


In [21]:
A3_mod.medium = mi7H9_OADCchol_Med

In [22]:
A3_sampFlux = sample(A3_mod, 5000)

In [23]:
A3_sampFlux.to_csv(samp_mi7H9OADCCholDir + 'A3' + '_cur.csv')

In [24]:
A4_mod = defModLst[12].copy()

In [25]:
A4_mod

Name,A4
Memory address,0x01b6789ff6d0
Number of metabolites,977
Number of reactions,1213
Number of groups,0
Objective expression,1.0*BIOMASS__2 - 1.0*BIOMASS__2_reverse_ecbec
Compartments,"c, e"


In [26]:
A4_mod.medium = mi7H9_OADCchol_Med

In [27]:
A4_sampFlux = sample(A4_mod, 5000)

In [28]:
A4_sampFlux.to_csv(samp_mi7H9OADCCholDir + 'A4' + '_cur.csv')

In [26]:
L9_rxns = [x.id for x in L9.reactions]

In [30]:
[x for x in list(mi7H9_OADC_Med.keys()) if x not in L9_rxns]

[]

In [32]:
sampleFluxes(defModLst, size = 5, outCsv = False, outDir = samp_mi7H10Dir, medium = mi7H9_OADC_Med)

Sampling L1 model...
Sampling L2 model...
Sampling L3 model...
Sampling L4 model...
Sampling L5 model...
Sampling L6 model...
Sampling L7 model...
Sampling L8 model...
Sampling L9 model...
Sampling A1 model...
Sampling A2 model...
Sampling A3 model...
Sampling A4 model...


{'L1':        ABTA          AGMT      ALAD_L         ARGDC     ARGSL     ARGSS  \
 0  0.000945 -1.864547e-21  902.493451 -1.839121e-21  0.000493  0.000493   
 1  0.000863 -1.861170e-21  912.936211 -1.835791e-21  0.002012  0.002012   
 2  0.000863  1.600912e-22  912.936704  1.858733e-22  0.000639  0.000639   
 3  0.003828 -3.063318e-18   97.445667 -3.021502e-18  0.023940  0.023940   
 4  0.007846 -3.779285e-16  100.117211 -3.778680e-16  0.019108  0.019108   
 
        ASNN     ASNS1    ASP1DC     ASPO5  ...          BACA        SDH1  \
 0  0.000006  0.000027  0.000023  0.000013  ...  2.655539e-12  110.927803   
 1  0.000006  0.000028  0.006197  0.000013  ...  2.655272e-12  100.445651   
 2  0.000006  0.000030  0.006086  0.000014  ...  5.076554e-12  100.373356   
 3  0.000263  0.000371  0.232482  0.000058  ...  2.505411e-11  897.582255   
 4  0.000262  0.000480  0.231999  0.000061  ...  1.272637e-10  897.738374   
 
         ATPM  DHFR_copy1  BiomassGrowth_sMtb  BIOMASS__2  BIOMASS__2.1 